In [10]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image
from fastai.vision.all import *

In [17]:
# Go to project root folder
%cd ..

/home


In [16]:
# Read config
import pprint

with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)

pprint.pprint(config)

FileNotFoundError: [Errno 2] No such file or directory: 'params.yaml'

In [5]:
train_df = pd.read_csv(config['data_load']['processed_train_data_path'])
test_df = pd.read_csv(config['data_load']['processed_test_data_path'])

In [7]:
train_df.head()

,label,img
0,3,PILImage mode=L size=28x28
1,6,PILImage mode=L size=28x28
2,2,PILImage mode=L size=28x28
3,2,PILImage mode=L size=28x28
4,13,PILImage mode=L size=28x28


In [9]:
def get_x(r): return r['img']
def get_y(r): return r['label']
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock), get_x = get_x, get_y=get_y)
dls = dblock.dataloaders(train_df)

FileNotFoundError: [Errno 2] No such file or directory: 'PILImage mode=L size=28x28'